In [1]:
from keras import backend as K
import numpy as np

def softmax2d(x):
    # Todo: figure out why we have to save x.shape as a tuple instead of using it directly
    # using it directly causes a keras problem

    old_shape=(-1,int(x.shape[1]),int(x.shape[2]),int(x.shape[3]))
    new_shape=(-1,1,int(x.shape[1])*int(x.shape[2]),old_shape[3])
    nsx = K.reshape(x,new_shape)
    e = K.exp(nsx - K.max(nsx, axis=-2, keepdims=True))
    s = K.sum(e, axis=-2, keepdims=True)
    ns = e / s
    ex = K.reshape(ns, old_shape)
    return ex


# returns x,y coordinates[0-1) of maximum value for each channel
# input should pass through softmax2d before input to this funciton
def softargmax(x):
    xc = K.variable(np.arange(int(x.shape[1]))/(int(x.shape[1])-1))
    x1 = K.variable(np.ones([x.shape[1]]))
    yc = K.variable(np.arange(int(x.shape[2]))/(int(x.shape[2])-1))
    y1 = K.variable(np.ones([x.shape[2]]))
    xx = K.dot(yc, x)
    xx = K.dot(x1, xx)
    xy = K.dot(y1, x)
    xy = K.dot(xc, xy)
    nc=K.stack([xx,xy],axis=-1)
    return nc

Using TensorFlow backend.


In [5]:
from keras.layers import Dense,Input,BatchNormalization,Dropout,Conv2D,AvgPool2D,Flatten,Lambda,Concatenate
from keras.models import Model
import keras
from keras.applications.vgg16 import VGG16

commonkwargs = {"activation": 'relu','padding': 'valid'}
height = 160
width = 320
image_size = [width, height]
classes = 2

#three input layers
img_robot_input_TL = Input(shape=(height, width, 3), name='image_robot_input_TL')
img_robot_input_TR = Input(shape=(height, width, 3), name='image_robot_input_TR')
img_robot_input_BL = Input(shape=(height, width, 3), name='image_robot_input_BL')
img_robot_input_BR = Input(shape=(height, width, 3), name='image_robot_input_BR')
img_classifier_input = Input(shape=(height, width, 3), name='image_classifier_input')
pos_input_TL = Input(shape=(2,), name='position_input_TL')
pos_input_TR = Input(shape=(2,), name='position_input_TR')
pos_input_BL = Input(shape=(2,), name='position_input_BL')
pos_input_BR = Input(shape=(2,), name='position_input_BR')
img_input = Input(shape=(height, width, 3), name='image_input')

#conv net and attention layers
x=Conv2D(64, (3,3), strides=(2, 2), **commonkwargs, name='block1_conv1')(img_input)
x=BatchNormalization(name='batch_norm1')(x)
x=Conv2D(64, (3,3), strides=(1, 1), **commonkwargs, name='block1_conv2')(x)
x=BatchNormalization(name='batch_norm2')(x)
x=Conv2D(32, (3,3), strides=(1, 1), **commonkwargs, name='block2_conv1')(x)
x=BatchNormalization(name='batch_norm3')(x)
x=Conv2D(32, (3,3), strides=(1, 1), **commonkwargs, name='block2_conv2')(x)
x=BatchNormalization(name='batch_norm4')(x)
x=Conv2D(16, (3,3), strides=(1, 1), **commonkwargs, name='block3_conv1')(x)
x = Lambda(softmax2d, name='image_softmax2d')(x)
x = Lambda(softargmax, name="expected_feature_location")(x)
x = Dropout(.25)(x)
feature_layer = Flatten(name='flattend_feature')(x)
sub_model = Model(img_input, feature_layer, name='feature_model')
sub_model.summary()
sub_model.get_layer('block1_conv1').set_weights(VGG16(weights='imagenet', 
                                                      include_top=False).get_layer('block1_conv1').get_weights())

#classifier layers
classifier_dense1 = Dense(32, activation='relu', name='classifier_dense1')(sub_model(img_classifier_input))
classifier_dense2 = Dense(16, activation='relu', name='classifier_dense2')(classifier_dense1)
classifier_output = Dense(classes, activation='softmax', name='classifier_output')(classifier_dense2)

#robot layers
robot_dense_TL = Dense(16, activation='relu', name='robot_dense_TL')(keras.layers.concatenate(
    [sub_model(img_robot_input_TL), pos_input_TL], name='feature_TL'))
robot_dense_TR = Dense(16, activation='relu', name='robot_dense_TR')(keras.layers.concatenate(
    [sub_model(img_robot_input_TR), pos_input_TR], name='feature_TR'))
robot_dense_BL = Dense(16, activation='relu', name='robot_dense_BL')(keras.layers.concatenate(
    [sub_model(img_robot_input_BL), pos_input_BL], name='feature_BL'))
robot_dense_BR = Dense(16, activation='relu', name='robot_dense_BR')(keras.layers.concatenate(
    [sub_model(img_robot_input_BR), pos_input_BR], name='feature_BR'))
robot_dense1 = Dense(16, activation='relu', name='robot_dense1')(keras.layers.concatenate(
    [robot_dense_TL, robot_dense_TR, robot_dense_BL, robot_dense_BR], name='concatenate_dense'))
robot_dense2 = Dense(16, activation='relu', name='robot_dense2')(robot_dense1)
robot_output = Dense(1, activation='linear', name='robot_output')(robot_dense2)

model = Model([img_robot_input_TL, img_robot_input_TR, img_robot_input_BL, img_robot_input_BR, pos_input_TL, 
               pos_input_TR, pos_input_BL, pos_input_BR, img_classifier_input], [robot_output, classifier_output])
model.compile(optimizer='adam', lr=3e-4, loss={'robot_output': 'mse', 'classifier_output': 'categorical_crossentropy'}, 
              loss_weights={'robot_output': 1, 'classifier_output': 0.5}, metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 160, 320, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 79, 159, 64)       1792      
_________________________________________________________________
batch_norm1 (BatchNormalizat (None, 79, 159, 64)       256       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 77, 157, 64)       36928     
_________________________________________________________________
batch_norm2 (BatchNormalizat (None, 77, 157, 64)       256       
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 155, 32)       18464     
_________________________________________________________________
batch_norm3 (BatchNormalizat (None, 75, 155, 32)       128       
__________

In [6]:
import pydot, graphviz
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [37]:
import os
directory_classifier = "./training/"
directory_robot = "./sep10_2017_LocateTip/training_data/4clear/"
classifier_1_data = [directory_classifier + 'peduncle/' + each for each in os.listdir(directory_classifier + 'peduncle')]
classifier_0_data = [directory_classifier + 'cane/' + each for each in os.listdir(directory_classifier + 'cane')]+[directory_classifier + 'grape/' + each for each in os.listdir(directory_classifier + 'grape')]+[directory_classifier + 'leaf/' + each for each in os.listdir(directory_classifier + 'leaf')]
robot_data = [directory_robot + each for each in os.listdir(directory_robot)]
print(len(classifier_0_data), len(classifier_1_data), len(robot_data))

662 674 161


In [62]:
import h5py
from PIL import Image
import random


def generate_arrays_from_file(robot_data, classifier_1_data, classifier_0_data, batch_size):
    if batch_size > 50:
        batch_size = 50
    while 1:
        Y_classifier = []
        X_classifier = []
        Y_robot = []
        X_robot_imgTL = []
        X_robot_imgTR = []
        X_robot_imgBL = []
        X_robot_imgBR = []
        X_robot_posTL = []
        X_robot_posTR = []
        X_robot_posBL = []
        X_robot_posBR = []
        with h5py.File(robot_data[random.randrange(len(robot_data))], 'r') as f:
            sampled_frames = random.sample(range(len(f['reward'])), batch_size)
            for i in sampled_frames:
                X_robot_posTL.append([f['cut_position_TL'][0], f['cut_position_TL'][1]])
                X_robot_posTR.append([f['cut_position_TR'][0], f['cut_position_TR'][1]])
                X_robot_posBL.append([f['cut_position_BL'][0], f['cut_position_BL'][1]])
                X_robot_posBR.append([f['cut_position_BR'][0], f['cut_position_BR'][1]])
                img = Image.fromarray(f['image_TL'][i], 'RGB')
                img = img.resize(image_size, Image.ANTIALIAS)
                img.show()
                X_robot_imgTL.append(img)
                X_robot_imgTR.append(f['image_TR'][i].resize(image_size, Image.ANTIALIAS))
                X_robot_imgBL.append(f['image_BL'][i].resize(image_size, Image.ANTIALIAS))
                X_robot_imgBR.append(f['image_BR'][i].resize(image_size, Image.ANTIALIAS))
                Y_robot.append(f['reward'][i])
        X_robot_posTL = np.array(X_robot_posTL).astype("float")
        X_robot_posTR = np.array(X_robot_posTR).astype("float")
        X_robot_posBL = np.array(X_robot_posBL).astype("float")
        X_robot_posBR = np.array(X_robot_posBR).astype("float")
        X_robot_imgTL = np.array(X_robot_imgTL).astype("float")
        X_robot_imgTR = np.array(X_robot_imgTR).astype("float")
        X_robot_imgBL = np.array(X_robot_imgBL).astype("float")
        X_robot_imgBR = np.array(X_robot_imgBR).astype("float")
        Y_robot = np.array(Y_robot).astype("float")
        for i in random.sample(range(len(classifier_1_data)), int(batch_size/2)):
            with Image.open(classifier_1_data[i]) as im:
                img = im.resize(image_size, Image.ANTIALIAS)
                X_classifier.append(np.array(img).astype("float"))
                Y_classifier.append([1, 0])
        for i in random.sample(range(len(classifier_0_data)), batch_size - int(batch_size/2)):
            with Image.open(classifier_0_data[i]) as im:
                img = im.resize(image_size, Image.ANTIALIAS)
                X_classifier.append(np.array(img).astype("float"))
                Y_classifier.append([0, 1])
        X_classifier = np.array(X_classifier).astype("float")
        Y_classifier = np.array(Y_classifier).astype("float")
        #yield (X, Y)
        return (X_classifier, Y_classifier, X_robot_imgTL, X_robot_imgTR, X_robot_imgBL, X_robot_imgBR,
                X_robot_posTL, X_robot_posTR, X_robot_posBL, X_robot_posBR)

In [63]:
(X_classifier, Y_classifier, X_robot_imgTL, X_robot_imgTR, X_robot_imgBL, X_robot_imgBR, X_robot_posTL, 
X_robot_posTR, X_robot_posBL, X_robot_posBR) = generate_arrays_from_file(robot_data, classifier_1_data, classifier_0_data, 10)

TypeError: 'list' object cannot be interpreted as an integer

In [52]:
print(X_robot_imgTL.shape)

(10, 400, 800, 3)
